In [1]:
# This is the Advanced GenAI Internship Hackathon notebook
# Name: Indhumathi
# Date: 31 Jan 2026

# Import necessary libraries
import pandas as pd
import sqlite3


In [2]:
# Uploading the files (orders.csv, users.json, restaurants.sql)
from google.colab import files

uploaded = files.upload()  # Click "Choose Files" and select all three files


Saving restaurants.sql to restaurants.sql
Saving users.json to users.json
Saving orders.csv to orders.csv


In [3]:
# Load CSV file
orders = pd.read_csv("orders.csv")

# Check first 5 rows to see the data
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
# Load JSON file
users = pd.read_json("users.json")

# Check first 5 rows
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
# Create an SQLite database in Colab
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

# Read SQL file
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

# Execute SQL commands to create the table and insert data
cursor.executescript(sql_script)
conn.commit()

# Load the restaurants table into a DataFrame
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# Check first 5 rows
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [6]:
# Merge orders with users on user_id
df = orders.merge(users, on="user_id", how="left")

# Merge the above result with restaurants on restaurant_id
df = df.merge(restaurants, on="restaurant_id", how="left")

# Check the merged dataset
df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
# Save the final dataset to CSV
df.to_csv("final_food_delivery_dataset.csv", index=False)

# Download the CSV (optional)
files.download("final_food_delivery_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
#1. City with highest total revenue from Gold members

# Check all columns in the final dataset
df.columns

# Filter only Gold members
gold_df = df[df['membership'] == 'Gold']

# Check the first 5 rows
gold_df.head()

# Group by city and sum total revenue
city_revenue = gold_df.groupby('city')['total_amount'].sum()

# Sort descending to find the city with highest revenue
city_revenue.sort_values(ascending=False)

# Display the top city
city_revenue.head(1)


,total_amount
city,
Bangalore,994702.59


In [15]:
# Question 2: Find the cuisine with highest average order value

# Check column names in dataset
print(df.columns)

# Group by cuisine and get average order value
avg_order_cuisine = df.groupby('cuisine')['total_amount'].mean()

# Sort from high to low
avg_order_cuisine_sorted = avg_order_cuisine.sort_values(ascending=False)

# Show all cuisines with their average order value
print(avg_order_cuisine_sorted)

# Show the top cuisine
top_cuisine = avg_order_cuisine_sorted.index[0]
top_value = avg_order_cuisine_sorted.iloc[0]
print("Top Cuisine:", top_cuisine, "→ Average Order Value:", top_value)


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')
cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64
Top Cuisine: Mexican → Average Order Value: 808.0213444401395


In [16]:
# Question 3: Count distinct users with total orders > 1000

# Step 1: Group by user and sum all their orders
user_total = df.groupby('user_id')['total_amount'].sum()

# Step 2: Filter users who spent more than 1000
users_over_1000 = user_total[user_total > 1000]

# Step 3: Count them
num_users = len(users_over_1000)

# Show the number of users
print("Number of users with total orders > 1000:", num_users)


Number of users with total orders > 1000: 2544


In [20]:
# Question 4: Which restaurant rating range generated the highest total revenue

# Make rating ranges
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

# Create new column for rating range using 'rating' column
df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels, include_lowest=True)

# Group by rating range and sum revenue
rating_revenue = df.groupby('rating_range')['total_amount'].sum()

# Sort descending to get highest
rating_revenue_sorted = rating_revenue.sort_values(ascending=False)

# Show revenue for all ranges
print(rating_revenue_sorted)

# Show the rating range with highest revenue
top_range = rating_revenue_sorted.index[0]
top_value = rating_revenue_sorted.iloc[0]
print("Rating range with highest total revenue:", top_range, "→ Revenue:", top_value)


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64
Rating range with highest total revenue: 4.6–5.0 → Revenue: 2197030.75


/tmp/ipython-input-836597442.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_revenue = df.groupby('rating_range')['total_amount'].sum()


In [21]:
# Question 5: Among Gold members, find the city with highest average order value

# Filter only Gold members
gold_df = df[df['membership'] == 'Gold']

# Group by city and calculate average order value
gold_avg_city = gold_df.groupby('city')['total_amount'].mean()

# Sort descending to get highest average order value
gold_avg_city_sorted = gold_avg_city.sort_values(ascending=False)

# Show average order value for each city
print(gold_avg_city_sorted)

# Show the top city
top_city = gold_avg_city_sorted.index[0]
top_value = gold_avg_city_sorted.iloc[0]
print("City with highest average order value among Gold members:", top_city, "→ Average Order Value:", top_value)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64
City with highest average order value among Gold members: Chennai → Average Order Value: 808.4590800299178


In [22]:
# Question 6: Cuisine with lowest number of distinct restaurants but still significant revenue

# Step 1: Count distinct restaurants per cuisine
restaurant_count = df.groupby('cuisine')['restaurant_id'].nunique()

# Step 2: Sum total revenue per cuisine
cuisine_revenue = df.groupby('cuisine')['total_amount'].sum()

# Step 3: Combine counts and revenue in a single table
summary = pd.DataFrame({
    'num_restaurants': restaurant_count,
    'total_revenue': cuisine_revenue
})

# Show the table
print(summary)

# Step 4: Sort by number of restaurants (ascending) and revenue (descending)
summary_sorted = summary.sort_values(by=['num_restaurants', 'total_revenue'], ascending=[True, False])

# Show the top cuisine
top_cuisine = summary_sorted.index[0]
print("Cuisine with lowest number of distinct restaurants but significant revenue:", top_cuisine)


         num_restaurants  total_revenue
cuisine                                
Chinese              120     1930504.65
Indian               126     1971412.58
Italian              126     2024203.80
Mexican              128     2085503.09
Cuisine with lowest number of distinct restaurants but significant revenue: Chinese


In [23]:
# Question 7: Percentage of total orders placed by Gold members

# Total number of orders
total_orders = len(df)

# Number of orders by Gold members
gold_orders = len(df[df['membership'] == 'Gold'])

# Calculate percentage and round
percentage_gold = round((gold_orders / total_orders) * 100)

# Show the result
print("Percentage of orders by Gold members:", percentage_gold, "%")


Percentage of orders by Gold members: 50 %


In [25]:
# Question 8: Restaurant with highest average order value but less than 20 total orders

# Group by restaurant name and calculate mean order value and count
restaurant_stats = df.groupby('restaurant_name_y').agg({
    'total_amount': ['mean', 'count']
})

# Filter restaurants with less than 20 orders
small_restaurants = restaurant_stats[restaurant_stats[('total_amount', 'count')] < 20]

# Sort by mean order value descending
small_restaurants_sorted = small_restaurants.sort_values(by=('total_amount', 'mean'), ascending=False)

# Show the top restaurant
top_restaurant = small_restaurants_sorted.index[0]  # This is the actual restaurant name
top_value = small_restaurants_sorted[('total_amount', 'mean')].iloc[0]

print("Restaurant with highest average order value (<20 orders):", top_restaurant)
print("Average Order Value:", top_value)


Restaurant with highest average order value (<20 orders): Restaurant_294
Average Order Value: 1040.2223076923076


In [28]:
# Question 9: Membership + Cuisine combination with highest total revenue

# Step 1: Group by membership and cuisine, sum total revenue
combo_revenue = df.groupby(['membership', 'cuisine'])['total_amount'].sum()

# Step 2: Sort descending to find the highest revenue combination
combo_revenue_sorted = combo_revenue.sort_values(ascending=False)

# Step 3: Show all combinations
print(combo_revenue_sorted)

# Step 4: Show the top combination
top_combo = combo_revenue_sorted.index[0]  # tuple: (membership, cuisine)
top_value = combo_revenue_sorted.iloc[0]

print("Highest revenue combination:", top_combo, "→ Revenue:", top_value)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64
Highest revenue combination: ('Regular', 'Mexican') → Revenue: 1072943.3


In [29]:
# Question 10: Find the quarter with highest total revenue

# Step 1: Make sure order_date is datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Step 2: Create a new column for quarter
df['quarter'] = df['order_date'].dt.quarter

# Step 3: Group by quarter and sum total revenue
quarter_revenue = df.groupby('quarter')['total_amount'].sum()

# Step 4: Sort descending to get highest revenue
quarter_revenue_sorted = quarter_revenue.sort_values(ascending=False)

# Show revenue per quarter
print(quarter_revenue_sorted)

# Step 5: Show the quarter with highest revenue
top_quarter = quarter_revenue_sorted.index[0]
top_value = quarter_revenue_sorted.iloc[0]
print("Quarter with highest total revenue: Q" + str(top_quarter), "→ Revenue:", top_value)


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64
Quarter with highest total revenue: Q3 → Revenue: 2037385.1


/tmp/ipython-input-563461.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


In [30]:
# Filter only Gold members
gold_orders_count = len(df[df['membership'] == 'Gold'])

# Show the answer
print("Total orders by Gold members:", gold_orders_count)


Total orders by Gold members: 4987


In [31]:
# Filter orders placed in Hyderabad
hyderabad_revenue = round(df[df['city'] == 'Hyderabad']['total_amount'].sum())

# Show the total revenue (rounded to nearest integer)
print("Total revenue from Hyderabad:", hyderabad_revenue)


Total revenue from Hyderabad: 1889367


In [32]:
# Count distinct users who placed at least one order
distinct_users = df['user_id'].nunique()

# Show the number of distinct users
print("Distinct users with at least one order:", distinct_users)


Distinct users with at least one order: 2883


In [33]:
# Calculate average order value for Gold members
avg_order_gold = round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)

# Show the result
print("Average order value for Gold members:", avg_order_gold)


Average order value for Gold members: 797.15


In [34]:
# Filter orders for restaurants with rating >= 4.5
orders_rating_45 = len(df[df['rating'] >= 4.5])

# Show the number of orders
print("Orders for restaurants with rating ≥ 4.5:", orders_rating_45)


Orders for restaurants with rating ≥ 4.5: 3374


In [35]:
# Filter only Gold members
gold_df = df[df['membership'] == 'Gold']

# Find the city among Gold members with highest total revenue
top_gold_city = gold_df.groupby('city')['total_amount'].sum().idxmax()

# Count how many orders were placed in that city
orders_top_gold_city = len(gold_df[gold_df['city'] == top_gold_city])

# Show the result
print("Orders in top revenue Gold city:", orders_top_gold_city)


Orders in top revenue Gold city: 1337
